In [2]:
%load_ext autoreload
%autoreload 2
import torch as th

In [4]:
import torch as th
import torch

In [42]:
def create_multi_hot_token_mask(tokens, target_words, seq_len=None):
    mask = th.zeros(len(tokens) if seq_len is None else seq_len, dtype=th.bool)
    for i, token in enumerate(tokens):
        # Remove the special prefix if present
        clean_token = token.replace('▁', '') if token.startswith('▁') else token
        if clean_token.endswith('</w>'):
            clean_token = clean_token.split('</w>')[0]
        if clean_token.startswith('<|') and clean_token.endswith('|>'):
            continue
        if clean_token in target_words:
            mask[i] = True
    return mask

In [6]:
from transformers import CLIPTextModelWithProjection, CLIPTokenizer
# Load SDXL's text encoder and tokenizer (text_encoder_2 and tokenizer_2)
text_encoder = CLIPTextModelWithProjection.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", subfolder="text_encoder_2", 
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",)
tokenizer = CLIPTokenizer.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", subfolder="tokenizer_2", 
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",)

/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:
prompt = 'blue circle is above and to the left of the red triangle'
# pipeline.
token_splits = tokenizer.tokenize(prompt,
                                  max_length=20,
                                padding='max_length',
                                truncation=True,
                                return_attention_mask=True,
                                add_special_tokens=True,
                                return_tensors='pt')
for text_targets in [["square"], 
                            ["triangle"], 
                            ["circle"], 
                            ["red"], 
                            ["blue"],
                            ["below", "left", "top", "right", "above"],
                            ["below", ],
                            ["left", ],
                            ["above", ],
                            ["right", ],
                            ["and", "to", "the", "of", "on", "is"],]:
    # text_targets = ["and", "to", "the", "of", "on", "is"]
    # template_type = f"image {imgtoken_type} to text {' '.join(text_targets)}"
    text_mask = create_multi_hot_token_mask(token_splits, text_targets, seq_len=20)
    print(text_mask.to(int).tolist())

Keyword arguments {'max_length': 20, 'padding': 'max_length', 'truncation': True, 'return_attention_mask': True, 'add_special_tokens': True} not recognized.


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
prompt = 'blue circle is above and to the left of the red triangle'
# pipeline.
text_tokens_and_mask = tokenizer(
        prompt,
        max_length=20,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
input_ids = text_tokens_and_mask['input_ids']
if input_ids.ndim == 2:
    input_ids = input_ids[0]
token_splits = [tokenizer.decode(i) for i in input_ids]
for text_targets in [["square"], 
                            ["triangle"], 
                            ["circle"], 
                            ["red"], 
                            ["blue"],
                            ["below", "left", "top", "right", "above"],
                            ["below", ],
                            ["left", ],
                            ["above", ],
                            ["right", ],
                            ["and", "to", "the", "of", "on", "is"],]:
    # text_targets = ["and", "to", "the", "of", "on", "is"]
    # template_type = f"image {imgtoken_type} to text {' '.join(text_targets)}"
    text_mask = create_multi_hot_token_mask(token_splits, text_targets, seq_len=20)
    print(text_mask.to(int).tolist())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [37]:
text_tokens_and_mask = tokenizer(
        prompt,
        max_length=20,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )

text_tokens_and_mask['input_ids'] = text_tokens_and_mask['input_ids']
text_tokens_and_mask['attention_mask'] = text_tokens_and_mask['attention_mask']

In [38]:
text_tokens_and_mask['input_ids']

tensor([[49406,  1746,  7117,   533,  4348,   537,   531,   518,  1823,   539,
           518,   736, 14615, 49407,     0,     0,     0,     0,     0,     0]])

In [40]:
text_tokens_and_mask['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])

In [39]:
for i in text_tokens_and_mask['input_ids'][0]:
    print(tokenizer.decode(i))

<|startoftext|>
blue
circle
is
above
and
to
the
left
of
the
red
triangle
<|endoftext|>
!
!
!
!
!
!


In [33]:
token_splits = tokenizer.tokenize(prompt,
                                  max_length=20,
                                padding='max_length',
                                truncation=True,
                                return_attention_mask=True,
                                add_special_tokens=True,
                                return_tensors='pt')

Keyword arguments {'max_length': 20, 'padding': 'max_length', 'truncation': True, 'return_attention_mask': True, 'add_special_tokens': True, 'return_tensors': 'pt'} not recognized.


In [36]:
tokenizer

CLIPTokenizer(name_or_path='stabilityai/stable-diffusion-xl-base-1.0', vocab_size=49408, model_max_length=77, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '!'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("!", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	49406: AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	49407: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

In [35]:
import inspect
print(inspect.getsource(tokenizer.tokenize))

    def tokenize(self, text: TextInput, **kwargs) -> list[str]:
        """
        Converts a string into a sequence of tokens, using the tokenizer.

        Split in words for word-based vocabulary or sub-words for sub-word-based vocabularies
        (BPE/SentencePieces/WordPieces). Takes care of added tokens.

        Args:
            text (`str`):
                The sequence to be encoded.
            **kwargs (additional keyword arguments):
                Passed along to the model-specific `prepare_for_tokenization` preprocessing method.

        Returns:
            `list[str]`: The list of tokens.
        """
        split_special_tokens = kwargs.pop("split_special_tokens", self.split_special_tokens)

        text, kwargs = self.prepare_for_tokenization(text, **kwargs)

        if kwargs:
            logger.warning(f"Keyword arguments {kwargs} not recognized.")

        if hasattr(self, "do_lower_case") and self.do_lower_case:
            # convert non-special tokens to lower

In [32]:
token_splits

['blue</w>',
 'circle</w>',
 'is</w>',
 'above</w>',
 'and</w>',
 'to</w>',
 'the</w>',
 'left</w>',
 'of</w>',
 'the</w>',
 'red</w>',
 'triangle</w>']

In [27]:
token_splits

['blue</w>',
 'circle</w>',
 'is</w>',
 'above</w>',
 'and</w>',
 'to</w>',
 'the</w>',
 'left</w>',
 'of</w>',
 'the</w>',
 'red</w>',
 'triangle</w>']

In [26]:
text_mask

tensor([False, False,  True, False,  True,  True,  True, False,  True,  True,
        False, False, False, False, False, False, False, False, False, False])

In [23]:
text_mask

tensor([False, False,  True, False,  True,  True,  True, False,  True,  True,
        False, False, False, False, False, False, False, False, False, False])

In [18]:
token_splits

['blue</w>',
 'circle</w>',
 'is</w>',
 'above</w>',
 'and</w>',
 'to</w>',
 'the</w>',
 'left</w>',
 'of</w>',
 'the</w>',
 'red</w>',
 'triangle</w>']

In [19]:
'blue</w>'.split('</w>')[0]

'blue'